In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_CONNECTIONS_CONFIG, SMOL_LM_135M

cfg = GRPO_CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

In [ ]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

In [ ]:
from model.reasoning import ConnectionsDataModule

In [ ]:
conn = ConnectionsDataModule(wrapper.tokenizer, wrapper.config)

In [ ]:
import itertools
import pandas as pd
dataset = pd.read_json("../dataset_files/connections_prompts.jsonl", lines=True)
df_groups = pd.json_normalize(dataset['solution'], 'groups')

group_list = [
    {
        "groups": (g := df_groups.sample(4, replace=False).reset_index(drop=True)).to_dict(orient="records"),
        "words": list(itertools.chain.from_iterable(g['words'].dropna()))
    }
    for _ in range(100000)
]

out = pd.DataFrame(group_list)

In [ ]:
out

In [ ]:
wrapper.init_data_module()

In [ ]:
wrapper.init_trainer()

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
wrapper.train()